Jupyter notebook is used over a regular .py file as we can keep datasets in memory between training runs

## Step 0. Import module and dataset files (if using Google Colab)

In [ ]:
# !git clone https://github.com/jckpn/age-gender-cnn

# from google.colab import drive
# drive.mount('/content/gdrive')

# !mkdir datasets
# !unzip -q gdrive/MyDrive/wiki_crop.zip -d datasets
# !unzip -q gdrive/MyDrive/imdb_crop_clean.zip -d datasets
# !unzip -q gdrive/MyDrive/adience.zip -d datasets
# !unzip -q gdrive/MyDrive/utkface.zip -d datasets
# !echo "Imported datasets!"

## 1. Import other modules

In [83]:
# Cite this
%load_ext autoreload
%autoreload 2

# Cite this
import sys
sys.path.insert(0, 'age-gender-cnn')

from train import train_model
from torch.utils.data import random_split, ConcatDataset
from torch.nn import CrossEntropyLoss, MSELoss
from face_dataset import *
from label_funcs import *
import preprocessor
from ds_transforms import *
from networks import *
import tests
from augmentations import get_augs

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Reading and processing 10000 files from C:\Users\jckpn\Documents\YEAR 3 PROJECT\implementation\source\other\imdb_crop into memory...


  0%|          | 1/10000 [00:05<14:30:12,  5.22s/it]

Equalising - 2500 entries per class (4 classes)



10000 items successfully prepared
Equalised datset to 2500 images per class
{'0': 2501, '1': 2501, '2': 2501, '3': 2497}

Split dataset into 8000 training and 2000 validation examples


## 2. Prepare datasets

In [ ]:
 Define dataset parameters
ds_size = 10000
processor = preprocessor.process(crop='mid', size=84)
transform = alexnet_transform(84)
print_errors = False

imdb_gender_ds = EqMemoryDataset(
    'C:\\Users\\jckpn\\Documents\\YEAR 3 PROJECT\\implementation\\source\\other\\imdb_crop',
    label_func=class_age_label(4), transform=transform, processor=processor,
    ds_size=ds_size, print_errors=print_errors)

train_val_set = imdb_gender_ds#ConcatDataset([wiki_gender_ds, imdb_gender_ds, utkface_gender_ds])

# Split dataset into training and validation sets
val_split_ratio = 0.2
val_size = int(val_split_ratio * len(train_val_set))
train_size = len(train_val_set) - val_size
train_set, val_set = random_split(train_val_set, [train_size, val_size])
print(f'Split dataset into {len(train_set)} training and {len(val_set)} validation examples')

test_set = val_set


## 3. Train model(s)

In [82]:
model = train_model(
    model=LeNet(2, in_channels=3), 

    train_set=train_set,
    val_set=val_set,
    image_resize=40,
    aug_transform=get_augs(),
    optim_fn=torch.optim.Adam,
    learning_rate=0.0001,

    patience=5,
    max_epochs=30,
    
    model_save_dir='./models')

c:\Users\jckpn\anaconda3\envs\fyp\lib\site-packages\torch\nn\modules\lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '



TRAINING MODEL LeNet-2_2904-0237.pt WITH PARAMS:
 - Architecture: LeNet
 - Learning rate: 0.0001
 - Optimizer: Adam
 - Loss function: CrossEntropyLoss()
 - Other notes: None

+---------------+---------------+---------------+---------------+---------------+
|         EPOCH | EXAMPLES SEEN |    TRAIN LOSS |      VAL LOSS |  ELAPSED TIME |
+---------------+---------------+---------------+---------------+---------------+


|             1 |           800 |        0.6943 |        0.6932 |          0:02 |


|             2 |          1600 |        0.6931 |        0.6918 |          0:04 |


|             3 |          2400 |        0.6914 |         0.691 |          0:05 |


...

| **         14 |         11200 |        0.6648 |        0.6843 |          0:24 |


| ***        15 |         12000 |        0.6594 |        0.6984 |          0:25 |


| ****       16 |         12800 |        0.6509 |        0.7131 |          0:27 |


| *****      17 |         13600 |        0.6648 |        0.6802 |          0:28 |
+---------------+---------------+---------------+---------------+---------------+

Halting training - 5 epochs without improvement

Training took 0m 28s (1.0s per epoch)

Best model from session saved to './models\LeNet-2_2904-0237.pt'

Testing class accuracy...



Confusion matrix (col: pred, row: actual/label):
          0          1
0    [       69][       31]
1    [       51][       49]

Accuracy per class:
Class 0: 69.00%
Class 1: 49.00%


## 4. Test model(s)

In [ ]:
tests.confusion_matrix(model, val_set, image_resize=40)